# Fouille données

In [ ]:
!pip install pandas numpy matplotlib seaborn folium scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from collections import Counter

%matplotlib inline
sns.set(style="whitegrid")

# Importation des données

In [ ]:
df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")
print("Nombre de photos au départ :  " + str(len(df)))

C:\Users\nputo\AppData\Local\Temp\ipykernel_34044\4015810889.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")


Nombre de photos au départ :  420240

Données nettoyées :


,id,user,lat,long,tags,title,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,2010,2,28,15,11
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,2010,2,28,17,51
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,2010,2,28,17,29
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,2010,1,28,20,15
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,2010,1,28,20,10


Nombre de photos restantes : 131973
Pourcentage de données supprimées : 68.60%


,id,user,lat,long,tags,title,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,2010,2,28,15,11
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,2010,2,28,17,51
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,2010,2,28,17,29
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,2010,1,28,20,15
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,2010,1,28,20,10
...,...,...,...,...,...,...,...,...,...,...,...
413846,48994653443,128086472@N06,45.781863,4.875416,NaN,L’égout et les couleurs,2019,10,30,16,42
419013,45039194824,146709606@N06,45.729498,4.951550,"car,autocar,interurbain,ligne,scolaire,yutong,...",YUTONG ICe 12 - DGC,2018,10,17,15,37
419071,44824515765,14869313@N00,45.768700,4.840502,"france,auvergnerhônealpes,auvergnerhonealpes,r...","Rhône river, Lyon, France",2018,10,27,11,41
419137,44995017704,48633948@N08,45.771852,4.833115,"lyon,1714,4814we69,man,hess,nmt222,trolleybus,...",Lyon 1714 Rue Pouteau,2017,10,18,14,20


# Nettoyage des données

In [45]:
df.columns = df.columns.str.strip()

df_clean = df.dropna(axis=1, how='all')

cols_date = ['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']

df_clean = df_clean.drop_duplicates(subset=['user', 'lat', 'long'] + cols_date)
df_clean = df_clean.dropna(subset=cols_date)

for col in cols_date:
    df_clean[col] = df_clean[col].astype(int)

colonnes_utiles = ['id', 'user', 'lat', 'long', 'tags', 'title'] + cols_date
df_clean = df_clean[colonnes_utiles]

print("\nDonnées nettoyées :")
display(df_clean.head())
print("Nombre de photos restantes : " + str(len(df_clean)))

pourcentage_supprimes = ((len(df) - len(df_clean)) / len(df)) * 100
print(f"Pourcentage de données supprimées : {pourcentage_supprimes:.2f}%")


Données nettoyées :


,id,user,lat,long,tags,title,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,2010,2,28,15,11
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,2010,2,28,17,51
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,2010,2,28,17,29
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,2010,1,28,20,15
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,2010,1,28,20,10


Nombre de photos restantes : 131973
Pourcentage de données supprimées : 68.60%


In [ ]:
print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")

df_clean = df_clean[
    (df_clean['date_taken_year'] >= 2004) &
    (df_clean['date_taken_year'] <= 2026)
]

print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")
print("Après filtre temporel : " + str(len(df_clean)) + " photos.")

In [ ]:
# Vérifier que les coordonnées géographiques sont cohérentes.
photos_avant_filtre = len(df_clean)

df_clean = df_clean[
    (df_clean['lat'] >= -90) & (df_clean['lat'] <= 90) &
    (df_clean['long'] >= -180) & (df_clean['long'] <= 180)
]
photos_supprimees = photos_avant_filtre - len(df_clean)
print(f"Photos supprimées par le filtre géographique : {photos_supprimees}")
print(f"Photos avec coordonnées valides : {len(df_clean)}")
print(f"Min/Max Latitude  : {df_clean['lat'].min()} / {df_clean['lat'].max()}")
print(f"Min/Max Longitude : {df_clean['long'].min()} / {df_clean['long'].max()}")

In [ ]:
from folium.plugins import FastMarkerCluster
# Création de la carte centrée sur une latitude et une longitude moyennes
m = folium.Map(location=[45.7640, 4.8357], zoom_start=13)
locations = list(zip(df['lat'], df['long']))
FastMarkerCluster(data=locations).add_to(m)
# Sauvegarde la carte dans un fichier HTML
m.save('ma_carte_lyon.html')
 

# Clustering